In [38]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

## Preparing the data

This part is already done in the previous notebooks. To know more details, please read them

In [39]:
housing = pd.read_csv("/home/jupyter/hands-on-ml/data/housing.csv")

In [40]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5,3.0, 4.5, 6., np.inf],
                               labels=[1,2,3,4,5])

In [41]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_data = housing.loc[train_index]
    strart_test_data = housing.loc[test_index]

In [ ]:
for set_ in (strat_train_data, strat_test_data):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
housing = strat_train_data.drop("median_house_value", axis=1)
housing_labels = strat_train_data["median_house_value"].copy()

In [ ]:
imputer = SimpleImputer(strategy="median")

In [44]:
housing_num = housing.drop("ocean_proximity", axis=1)

In [45]:
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix]/X[:, households_ix]
        population_per_household = X[:, population_ix]/X[:, households_ix]
        
        if self.add_bedrooms_per_room:
            beedroms_per_room = X[:, bedrooms_ix]/X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, beedroms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [46]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler()),
])

In [47]:
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [48]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

In [49]:
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

In [50]:
housing_prepared = full_pipeline.fit_transform(housing)

## Training and Evaluating on the Training Set

In [51]:
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Let's try it out on a few instances from the training set

In [52]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

In [53]:
print("Predictions: ", lin_reg.predict(some_data_prepared))
print("Labels: ", list(some_labels))

Predictions:  [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
Labels:  [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


It works, although the predictions are not exactly accurate. Let's measure this regression model's RMSE on the whole training set using Scikit-Learn's mean_sqaured_error() function

In [54]:
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

68628.19819848922


This is better than nothing, but clearly not a great score. This is an example of model underfitting. Let's try a DecisionTreeRegressor. This is a powerful model, capable of finding complex nonlinear relationship in the data.

In [55]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

Let's evaluate it on the training set:

In [56]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

0.0


Could this model really be absolutely perfect? Of course, but it is much more likely that the model has badly overfit the data. We need the model to predict on unseen data but, remember, don't touch the test set!!! Let's use the cross-validation to do it.

## Better Evaluation Using Cross-Validation

In [57]:
scores = cross_val_score(tree_reg, 
                         housing_prepared, 
                         housing_labels,
                         scoring="neg_mean_squared_error",
                         cv=10)

In [58]:
tree_rmse_scores = np.sqrt(-scores)

Scikit-Learn's cross validation features expect a utility function (greater is better) rather than a cost function (lower is better), so the scoring function is actually the opossite of the MSE (i.e, a negative value), which is why the preceding code computes -scores before calculating the sqaured root.

In [59]:
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("sd: ", scores.std())

In [60]:
display_scores(tree_rmse_scores)

Scores:  [69605.99157568 67495.50822143 71770.98282851 69548.34918194
 70256.24286126 74974.5217272  70767.01790763 71585.54690822
 76991.12331439 68656.89435874]
Mean:  71165.21788849903
sd:  2734.671887423419


Now, the Decision Tree does not look as good as it did earlier. In fact, it seems to perform worse than the Linear Regression model.Let's compute the same scores for the Linear Regression model.

In [61]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores:  [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
Mean:  69052.46136345083
sd:  2731.674001798348


Thanks to CV we can conclude that, Decission Tree model performs worse than Linear Regression. 

Let's try one last model now: the RandomForestRegressor. 

In [62]:
forest_reg = RandomForestRegressor()
random_forest_scores = cross_val_score(forest_reg, 
                         housing_prepared, 
                         housing_labels,
                         scoring="neg_mean_squared_error",
                         cv=10)

In [63]:
forest_rmse = np.sqrt(-random_forest_scores)
display_scores(forest_rmse)

Scores:  [49794.72793079 47383.22022651 49955.20397014 52253.02267178
 49557.63727495 53975.52583057 48788.170888   48029.83051955
 53021.02701237 50183.3017142 ]
Mean:  50294.16680388556
sd:  2039.9323881066575


Random Forest look very promising!!!

We should save every model we experiment with so that we can come back to any model we want. We have to make sure we save both the hyperparameters and the trained parameters, as well as the cross-validation scores and perhaps the actual predictions as well.

We can easily save Scikit-Learn models by using Python's pickle module or by using the joblib library, which is more efficient at serializing large NumPy arrays.

In [64]:
joblib.dump(forest_reg, "forest_reg.pkl")

['forest_reg.pkl']

## Grid search

We are going to use Scikit Learn's GridSearchCV to search for the value of the hyperparameters. All we neet to do is to tell it which hyperparameters we want it to experiment with and what values to try out, and it will use cross-validation to evaluate all the possible combinations of hyperparameters values. For example, the following code searches for the best combination of hyperparameters values for the RandomForestRegressor.

In [65]:
param_grid = [
    {'n_estimators' : [3, 10, 30], 'max_features' : [2, 4, 6, 8]},
    {'bootstrap' : [False], 'n_estimators' : [3, 10], 'max_features' : [2, 3, 4]}
]

In [66]:
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg,
                           param_grid,
                           scoring='neg_mean_squared_error',
                          return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n

Getting the best combination of parameters

In [67]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

We can also get the best estimator directly:

In [68]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=8, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=30, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

If GridSearchCV is initialized with refit=True (which is the default), then once it finds the best estimator using cross-validation, it retrains it on the whole training set. This is usually a good idea, since feeding it more data will likely improve its performance.

And of course the evaluation scores are also available

In [69]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

64433.431097937835 {'max_features': 2, 'n_estimators': 3}
55989.390499239096 {'max_features': 2, 'n_estimators': 10}
53048.4918218836 {'max_features': 2, 'n_estimators': 30}
60017.40671903815 {'max_features': 4, 'n_estimators': 3}
52746.59478774373 {'max_features': 4, 'n_estimators': 10}
50314.14096220161 {'max_features': 4, 'n_estimators': 30}
59049.568151840765 {'max_features': 6, 'n_estimators': 3}
52298.3318407487 {'max_features': 6, 'n_estimators': 10}
50286.288556011896 {'max_features': 6, 'n_estimators': 30}
59378.46457082463 {'max_features': 8, 'n_estimators': 3}
51898.15274683964 {'max_features': 8, 'n_estimators': 10}
50030.77844748372 {'max_features': 8, 'n_estimators': 30}
63986.63360619485 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54517.09155607244 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
60522.99489587505 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52940.56967662291 {'bootstrap': False, 'max_features': 3, 'n_estimators': 

## Analyze the Best Models and Their Errors

You will often gain good insights on the problem by inpecting the best models. For example, the RandomForestRegressor can indicate the relative importance of each attribute for making accurate predictions

In [70]:
feature_importance = grid_search.best_estimator_.feature_importances_
feature_importance

array([6.97095136e-02, 6.23705835e-02, 4.46203932e-02, 1.56691933e-02,
       1.56123510e-02, 1.47511402e-02, 1.40639406e-02, 3.75396703e-01,
       6.17975764e-02, 1.10181723e-01, 4.79504727e-02, 1.30995807e-02,
       1.49966303e-01, 5.09436073e-05, 1.62128171e-03, 3.13830031e-03])

Let's display these importance scores next to their corresponding attribute names

In [71]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importance, attributes), reverse = True)

[(0.3753967031945825, 'median_income'),
 (0.14996630253497126, 'INLAND'),
 (0.11018172345318232, 'pop_per_hhold'),
 (0.06970951357221002, 'longitude'),
 (0.062370583451518066, 'latitude'),
 (0.061797576417095754, 'rooms_per_hhold'),
 (0.04795047270007841, 'bedrooms_per_room'),
 (0.044620393238695606, 'housing_median_age'),
 (0.01566919333693217, 'total_rooms'),
 (0.01561235099410598, 'total_bedrooms'),
 (0.014751140170823813, 'population'),
 (0.014063940639910863, 'households'),
 (0.013099580665519734, '<1H OCEAN'),
 (0.003138300309664171, 'NEAR OCEAN'),
 (0.0016212817134146163, 'NEAR BAY'),
 (5.094360729463347e-05, 'ISLAND')]

With this information, we may want to try dropping some of the less useful features.

## Evaluate the system on the Test Set

In [76]:
final_model = grid_search.best_estimator_

X_test = strat_test_data.drop("median_house_value", axis=1)
y_test = strat_test_data["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

In [81]:
final_prediction = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_prediction)
final_rmse = np.sqrt(final_mse)

In [82]:
final_rmse

47881.683478456594